In [1]:
# Downloading the dataset
! python -m wget "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip"

e:\Anaconda\envs\tf2\python.exe: No module named wget


In [2]:
import zipfile
with zipfile.ZipFile('rps.zip', 'r') as zip_ref:
    zip_ref.extractall()


In [3]:
# Importing the inception V3 model
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Downloading the weights
import urllib

weights = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

weights_file = 'inception_v3.h5'
urllib.request.urlretrieve(weights, weights_file)

In [4]:
# Loading the weights
pretrained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None
)

pretrained_model.load_weights(weights_file)

In [5]:
for layer in pretrained_model.layers:
    layer.trainable = False

last_layer = pretrained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [6]:
from tensorflow.keras.layers import Flatten, Dense

x = Flatten()(last_output)
x = Dense(1024, activation='relu')(x)
x = Dense(3, activation='softmax')(x)

In [7]:
from tensorflow.keras import Model

model = Model(pretrained_model.input, x)

In [8]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.0001),metrics=['accuracy'])

In [9]:
# import image datagenerator

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
training_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [11]:
training_generator = training_datagen.flow_from_directory('rps', target_size=(150, 150), batch_size=32, class_mode='categorical')

Found 2520 images belonging to 3 classes.


In [13]:
history = model.fit(training_generator,epochs=25,verbose=1)

Epoch 1/25
79/79 [==============================] - 16s 204ms/step - loss: 0.1753 - accuracy: 0.9437
Epoch 2/25
79/79 [==============================] - 12s 151ms/step - loss: 0.0914 - accuracy: 0.9655
Epoch 3/25
79/79 [==============================] - 12s 157ms/step - loss: 0.0748 - accuracy: 0.9770
Epoch 4/25
79/79 [==============================] - 13s 168ms/step - loss: 0.0583 - accuracy: 0.9813
Epoch 5/25
79/79 [==============================] - 12s 145ms/step - loss: 0.0707 - accuracy: 0.9802
Epoch 6/25
79/79 [==============================] - 12s 145ms/step - loss: 0.0544 - accuracy: 0.9829
Epoch 7/25
79/79 [==============================] - 12s 151ms/step - loss: 0.0407 - accuracy: 0.9873
Epoch 8/25
79/79 [==============================] - 12s 147ms/step - loss: 0.0336 - accuracy: 0.9889
Epoch 9/25
79/79 [==============================] - 12s 148ms/step - loss: 0.0223 - accuracy: 0.9925
Epoch 10/25
79/79 [==============================] - 12s 148ms/step - loss: 0.0400 - accura